In [1]:
import anndata
import os
import requests

save_path = "data/example_sce.h5ad"
if not os.path.exists(save_path):
    response = requests.get("https://go.wisc.edu/69435h")
    with open(save_path, "wb") as f:
        f.write(response.content)

example_sce = anndata.read_h5ad(save_path)
example_sce

AnnData object with n_obs × n_vars = 2087 × 100
    obs: 'clusters_coarse', 'clusters', 'S_score', 'G2M_score', 'cell_type', 'sizeFactor', 'pseudotime'
    var: 'highly_variable_genes'
    uns: 'X_name', 'clusters_coarse_colors', 'clusters_colors', 'day_colors', 'neighbors', 'pca'
    obsm: 'PCA', 'UMAP', 'X_pca', 'X_umap'
    layers: 'counts', 'cpm', 'logcounts', 'spliced', 'unspliced'
    obsp: 'connectivities', 'distances'

In [2]:
from scdesigner.experimental.estimators import negative_binomial_regression
from scdesigner.experimental.transform import nullify, amplify, data_frame_mask

params = negative_binomial_regression(example_sce, "~ bs(pseudotime, degree=6)", epochs=100)
outcomes = example_sce.var_names[:4]
mask = data_frame_mask(params["coefficient"], ["pseudotime"], outcomes)
null_params = nullify(params, "coefficient", mask)
null_params["coefficient"]

,Pyy,Iapp,Chgb,Rbp4,Spp1,Chga,Cck,Ins1,Nnat,Ins2,...,Nkx6-1,Fxyd3,Hn1,Smarcd2,Pdia6,Ffar2,Hes6,Serpinh1,Npy,1110012L19Rik
Intercept,2.197865,2.268209,1.318629,1.286049,3.840294,0.448331,-0.349397,2.105087,1.068234,1.211985,...,0.552880,0.290994,1.440957,-0.160437,1.048037,-1.233168,-0.637515,1.724128,-2.435935,-0.178139
"bs(pseudotime, degree=6)[1]",0.000000,0.000000,0.000000,0.000000,3.003995,-15.069673,-1.397554,-10.418838,-11.106424,-13.283100,...,-1.553560,-3.440922,-1.226781,1.312675,0.608415,3.222896,3.997243,-0.758272,-5.780035,-3.938097
"bs(pseudotime, degree=6)[2]",0.000000,0.000000,0.000000,0.000000,-3.633759,4.878798,6.335586,1.287712,10.443920,-2.103594,...,3.010540,2.766718,3.452707,4.106813,0.911357,-0.469060,3.836551,1.942890,-2.752351,3.260376
"bs(pseudotime, degree=6)[3]",0.000000,0.000000,0.000000,0.000000,-14.695806,5.241379,5.936082,-6.154892,-7.455820,-6.045755,...,1.158814,2.801613,0.697978,1.391438,-4.358403,4.425304,0.441013,-1.976071,-3.182852,4.897079
"bs(pseudotime, degree=6)[4]",0.000000,0.000000,0.000000,0.000000,-1.076773,2.775726,2.266021,-8.846365,-4.309461,-8.769652,...,-0.812769,-1.884889,-4.701144,-2.436549,-1.081311,3.203225,-0.545738,-5.097927,-4.957916,-1.141697
"bs(pseudotime, degree=6)[5]",0.000000,0.000000,0.000000,0.000000,-5.658473,1.579606,-1.084344,1.496725,3.157530,2.442446,...,1.184013,2.084829,0.848783,0.073521,0.972107,0.687638,2.092434,-1.340444,-3.438744,-1.626804
"bs(pseudotime, degree=6)[6]",0.000000,0.000000,0.000000,0.000000,-4.979625,2.345560,1.942488,3.060781,3.689172,3.768801,...,1.268168,-0.480219,-1.241686,-0.989558,1.673595,1.211747,0.363372,-2.069894,6.488352,-0.483170


In [3]:
from scdesigner.experimental.samplers import negative_binomial_regression_sample

samples = negative_binomial_regression_sample(params, example_sce.obs, "~ bs(pseudotime, degree=6)")
samples.X[:10, :10]

array([[ 58,   1,  17,  18,   0,  44,  11,   0,   1,   0],
       [  0,   0,   7,   0,   0,   7,  13,   0,   0,   0],
       [141,  14,  19,  14,   0,   7,   0,   3,   2,   3],
       [ 11, 139,  21,  86,   0,  13,   1,  29,  16,  54],
       [  0,   0,   0,   0,   2,   2,  11,   0,   0,   0],
       [  0,   0,   0,   1,   7,   0,  12,   0,   1,   0],
       [  2,   0,   0,   0,  74,   0,   0,   0,   0,   1],
       [ 37,   6,  66,  27,   1,  52,   6,   1,   0,   0],
       [247,  27,  50, 105,   0,   4,   6,   0,   8,   4],
       [ 17,   2, 102,   6,   0,  23,  12,   0,   0,   1]])

In [ ]:
from scdesigner.experimental.diagnose import compare_means, compare_standard_deviation, compare_umap
import numpy as np

log_relative = lambda x: np.log1p(x / x.sum(axis=1, keepdims=True))
compare_means(example_sce, samples, log_relative)


alt.LayerChart(...)

In [5]:
compare_standard_deviation(example_sce, samples, log_relative)

alt.LayerChart(...)

In [6]:
compare_umap(example_sce, samples, transform=log_relative)

alt.FacetChart(...)

In [7]:
from scdesigner.experimental.estimators import negative_binomial_copula

params = negative_binomial_copula(example_sce, "~ bs(pseudotime, degree=2)", epochs=10)
mask = data_frame_mask(params["covariance"], outcomes)
null_params = nullify(params, "covariance", mask)
null_params["covariance"]

,Pyy,Iapp,Chgb,Rbp4,Spp1,Chga,Cck,Ins1,Nnat,Ins2,...,Nkx6-1,Fxyd3,Hn1,Smarcd2,Pdia6,Ffar2,Hes6,Serpinh1,Npy,1110012L19Rik
Pyy,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Iapp,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Chgb,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Rbp4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Spp1,0.386956,0.803187,-0.285239,0.294909,2.397775,-0.233185,-0.913331,0.561250,0.404071,0.330800,...,-0.097225,-0.077447,-0.024432,-0.202395,0.404236,-0.031295,-0.108697,0.033156,-0.139434,0.000094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ffar2,0.013964,0.023583,0.231922,0.000385,-0.031295,0.132574,-0.002764,0.023750,0.019366,0.039196,...,0.020851,0.089298,-0.038255,0.020226,0.012687,0.840218,-0.005212,-0.019902,-0.021411,0.055494
Hes6,0.316311,0.654256,-0.109340,0.171797,-0.108697,-0.072250,-0.037895,0.377461,0.380303,0.353003,...,0.157830,-0.028166,0.414552,0.411081,0.173491,-0.005212,1.169303,0.344088,-0.030760,0.075310
Serpinh1,0.438908,0.652418,-0.039318,0.286530,0.033156,0.019468,-0.165264,0.394605,0.379093,0.377717,...,0.094459,-0.002303,0.291651,0.347598,0.156644,-0.019902,0.344088,1.056873,-0.039356,0.091525
Npy,-0.411190,-0.255074,-0.152336,-0.301475,-0.139434,-0.105846,0.240795,0.070377,-0.025070,0.042381,...,0.088492,-0.018009,0.023997,-0.041064,0.059283,-0.021411,-0.030760,-0.039356,0.438286,-0.030924


In [8]:
mask = data_frame_mask(params["coefficient"], ["pseudotime"], outcomes)
null_params = amplify(params, "coefficient", mask, factor=2)
null_params["coefficient"] / params["coefficient"]

,Pyy,Iapp,Chgb,Rbp4,Spp1,Chga,Cck,Ins1,Nnat,Ins2,...,Nkx6-1,Fxyd3,Hn1,Smarcd2,Pdia6,Ffar2,Hes6,Serpinh1,Npy,1110012L19Rik
Intercept,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
"bs(pseudotime, degree=2)[1]",2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
"bs(pseudotime, degree=2)[2]",2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [9]:
from scdesigner.experimental.samplers import negative_binomial_copula_sample

negative_binomial_copula_sample(params, example_sce.obs, "~ bs(pseudotime, degree=2)")

AnnData object with n_obs × n_vars = 2087 × 100
    obs: 'clusters_coarse', 'clusters', 'S_score', 'G2M_score', 'cell_type', 'sizeFactor', 'pseudotime'